In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

# Define the input shape
img_rows, img_cols = 224, 224

# Path to the training and testing directories
train_path = 'Training'
test_path = 'Testing'

# Preprocessing step
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_path,
                                                    target_size=(img_rows, img_cols),
                                                    batch_size=32,
                                                    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_path,
                                                  target_size=(img_rows, img_cols),
                                                  batch_size=32,
                                                  class_mode='categorical')

# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add new layers for fine-tuning
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

# Combine the base model with the new layers
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator,
          epochs=3,
          validation_data=test_generator)

# Save the model
model.save('modelpretrained.h5')


Found 5229 images belonging to 4 classes.
Found 0 images belonging to 0 classes.
Epoch 1/3
164/164 [==============================] - 466s 3s/step - loss: 0.7942 - accuracy: 0.7653
Epoch 2/3
164/164 [==============================] - 462s 3s/step - loss: 0.3421 - accuracy: 0.8723
Epoch 3/3
164/164 [==============================] - 463s 3s/step - loss: 0.2645 - accuracy: 0.9011


In [3]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.models import load_model
from PIL import Image

# Path to the testing folder containing images to classify
test_path = 'Testing'

#Path to the trained model
model_path = 'modelpretrained.h5'

# Load the model
model = load_model(model_path)

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['ID', 'Label'])

# Loop over each image in the testing folder
for img_file in os.listdir(test_path):
    try:
        # Load the image and convert it to a numpy array
        img = Image.open(os.path.join(test_path, img_file))
        img = img.resize(( 224, 224))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)

        # Normalize the image
        x /= 255.0

        # Make a prediction
        pred = model.predict(x)

        # Get the predicted label
        label = np.argmax(pred)

        # Add the results to the DataFrame
        results_df = pd.concat([results_df, pd.DataFrame({'ID': [img_file], 'Label': [label]})], ignore_index=True)

    except Exception as e:
        print(f"Error processing image {img_file}: {e}")
        results_df = pd.concat([results_df, pd.DataFrame({'ID': [img_file], 'Label': [0]})], ignore_index=True)

# Print the results
print(results_df)


1/1 [==============================] - 0s 105ms/step
Error processing image 0362.jpg: Graph execution error:

Detected at node 'model/block1_conv1/Relu' defined at (most recent call last):
    File "C:\Users\MSI\anaconda3\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\MSI\anaconda3\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\MSI\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\MSI\anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\MSI\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\MSI\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\MSI\anaconda3\lib\asyncio\base_events.py",

1/1 [==============================] - 0s 101ms/step


1/1 [==============================] - 0s 97ms/step
           ID Label
0    0001.jpg     0
1    0002.jpg     2
2    0005.jpg     3
3    0006.jpg     1
4    0008.jpg     3
..        ...   ...
475  0992.jpg     3
476  0993.jpg     1
477  0996.jpg     0
478  0998.jpg     2
479  0999.jpg     3

[480 rows x 2 columns]


In [4]:
results_df['ID'] = results_df['ID'].str.replace('.jpg', '')

C:\Users\MSI\AppData\Local\Temp\ipykernel_4920\302816169.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  results_df['ID'] = results_df['ID'].str.replace('.jpg', '')


In [5]:
results_df.to_csv('submission.csv', index=False)